In [11]:
my_ProjectID = "medical-imaging-ai"

In [12]:
import os

PROJECT_ID = ""

if not os.getenv("IS_TESTING"):
    # Get your Google Cloud project ID from gcloud
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  medical-imaging-ai


In [13]:
import numpy as np
import pandas as pd


In [23]:
# CT image sample
#my_SeriesInstanceUID = "1.3.6.1.4.1.32722.99.99.232988001551799080335895423941323261228"

# pathology image sample
my_SeriesInstanceUID = "1.3.6.1.4.1.5962.99.1.219702714.188810011.1640897242554.2.0"

In [6]:
%%bigquery idc_summary_df --project=$my_ProjectID 

SELECT
  COUNT(DISTINCT(PatientID)) AS total_patients,
  COUNT(DISTINCT(StudyInstanceUID)) AS total_studies,
  COUNT(DISTINCT(SeriesInstanceUID)) AS total_series,
  COUNT(DISTINCT(SOPInstanceUID)) AS total_instances,
  COUNT(DISTINCT(collection_id)) AS total_collections,
  SUM(instance_size)/(POW(1024,4)) AS total_size_TB
FROM
  `bigquery-public-data.idc_current.dicom_all`

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.39s/rows]


In [7]:
idc_summary_df

,total_patients,total_studies,total_series,total_instances,total_collections,total_size_TB
0,64344,138557,468745,39142908,135,37.764894


In [24]:
from google.cloud import bigquery
bq_client = bigquery.Client(my_ProjectID)

selection_query = f"\
  SELECT  \
    StudyInstanceUID, \
    SeriesInstanceUID, \
    SOPInstanceUID, \
    gcs_url \
  FROM \
    `bigquery-public-data.idc_current.dicom_all` \
  WHERE \
    SeriesInstanceUID = \"{my_SeriesInstanceUID}\""

selection_result = bq_client.query(selection_query)
selection_df = selection_result.result().to_dataframe()

In [25]:
pd.set_option('display.max_colwidth', None)

selection_df

,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,gcs_url
0,2.25.16181252012499544165879445836446987048,1.3.6.1.4.1.5962.99.1.219702714.188810011.1640897242554.2.0,1.3.6.1.4.1.5962.99.1.219702714.188810011.1640897242554.35.0,gs://idc-open-idc1/ff2af67d-5f79-4717-b7f4-8028684436dd.dcm
1,2.25.16181252012499544165879445836446987048,1.3.6.1.4.1.5962.99.1.219702714.188810011.1640897242554.2.0,1.3.6.1.4.1.5962.99.1.219702714.188810011.1640897242554.7.0,gs://idc-open-idc1/fd57f940-f2e5-4898-9989-1f666cac3a09.dcm
2,2.25.16181252012499544165879445836446987048,1.3.6.1.4.1.5962.99.1.219702714.188810011.1640897242554.2.0,1.3.6.1.4.1.5962.99.1.219702714.188810011.1640897242554.21.0,gs://idc-open-idc1/3ba34f9b-b657-4292-b041-834565635e97.dcm
3,2.25.16181252012499544165879445836446987048,1.3.6.1.4.1.5962.99.1.219702714.188810011.1640897242554.2.0,1.3.6.1.4.1.5962.99.1.219702714.188810011.1640897242554.14.0,gs://idc-open-idc1/ca35c29d-405b-4ca1-aadb-e831414ae2b6.dcm
4,2.25.16181252012499544165879445836446987048,1.3.6.1.4.1.5962.99.1.219702714.188810011.1640897242554.2.0,1.3.6.1.4.1.5962.99.1.219702714.188810011.1640897242554.28.0,gs://idc-open-idc1/c92c2940-1033-4cfe-9200-7f733add7f38.dcm


In [26]:
# save the list of files
# save the list of GCS URLs into a file
import os
idc_download_folder = "./IDC_downloads"
if not os.path.exists(idc_download_folder):
  os.mkdir(idc_download_folder)

selection_manifest = os.path.join(idc_download_folder, "idc_manifest.txt")
selection_df["gcs_url"].to_csv(selection_manifest, header=False, index=False)

In [28]:
# copy files to staging area
# download is this simple!
!cat ./IDC_downloads/idc_manifest.txt | gsutil -m cp -I gs://miai-staging_bucket/CPTAC-LSCC

Copying gs://idc-open-idc1/ff2af67d-5f79-4717-b7f4-8028684436dd.dcm [Content-Type=application/dicom]...
Copying gs://idc-open-idc1/fd57f940-f2e5-4898-9989-1f666cac3a09.dcm [Content-Type=application/dicom]...
Copying gs://idc-open-idc1/3ba34f9b-b657-4292-b041-834565635e97.dcm [Content-Type=application/dicom]...
Copying gs://idc-open-idc1/c92c2940-1033-4cfe-9200-7f733add7f38.dcm [Content-Type=application/dicom]...
Copying gs://idc-open-idc1/ca35c29d-405b-4ca1-aadb-e831414ae2b6.dcm [Content-Type=application/dicom]...
\ [5/5 files][183.3 MiB/183.3 MiB] 100% Done                                    
Operation completed over 5 objects/183.3 MiB.                                    


In [29]:
!gsutil ls gs://miai-staging_bucket/CPTAC-LSCC

gs://miai-staging_bucket/CPTAC-LSCC/3ba34f9b-b657-4292-b041-834565635e97.dcm
gs://miai-staging_bucket/CPTAC-LSCC/c92c2940-1033-4cfe-9200-7f733add7f38.dcm
gs://miai-staging_bucket/CPTAC-LSCC/ca35c29d-405b-4ca1-aadb-e831414ae2b6.dcm
gs://miai-staging_bucket/CPTAC-LSCC/fd57f940-f2e5-4898-9989-1f666cac3a09.dcm
gs://miai-staging_bucket/CPTAC-LSCC/ff2af67d-5f79-4717-b7f4-8028684436dd.dcm
